# Create and Edit Azure Search enrichment resources
### Run this notebook once you have completed the following steps
  * Run the K-Means clustering skill
  * Updated the Search index with the centroid field
  
### This notebook will create a new data source and indexer to update the index with the centroid field value

In [3]:
import numpy as np
import requests
import json
import pandas as pd
import io

In [5]:
search_service_name = 'customerdemos'
search_service = f"https://{search_service_name}.search.windows.net"
api_version = '2019-05-06-Preview'
api_key = ''
content_type = 'application/json'
cognitiveServicesKey = ''
STORAGEACCOUNTNAME= 'knowstoredemo'
STORAGEACCOUNTKEY= ''
knowledge_store_key = f'DefaultEndpointsProtocol=https;AccountName={STORAGEACCOUNTNAME};AccountKey={STORAGEACCOUNTKEY};EndpointSuffix=core.windows.net'


def construct_Url(service, resource, resource_name, action, api_version):
    if resource_name:
        
        if action:
            return service + '/'+ resource + '/' + resource_name + '/' + action + '?api-version=' + api_version
        else:
            return service + '/'+ resource + '/' + resource_name + '?api-version=' + api_version
    else:
        return service + '/'+ resource + '?api-version=' + api_version


headers = {'api-key': api_key, 'Content-Type': content_type}
print(construct_Url(search_service, "indexes", "caselaw", "analyze", api_version))
print(construct_Url(search_service, "indexes", "caselaw", None, api_version))
print(construct_Url(search_service, "indexers", None, None, api_version))


https://customerdemos.search.windows.net/indexes/caselaw/analyze?api-version=2019-05-06-Preview
https://customerdemos.search.windows.net/indexes/caselaw?api-version=2019-05-06-Preview
https://customerdemos.search.windows.net/indexers?api-version=2019-05-06-Preview


In [4]:
# r = requests.get(construct_Url(search_service, "indexers", "yourindexername", None, api_version), headers=headers)
# res = r.content
# resdata = r.json() 
# print(json.dumps(resdata, indent=2))
print("Commented out sections not needed for this demo")

{
  "@odata.context": "https://customerdemos.search.windows.net/$metadata#indexers/$entity",
  "@odata.etag": "\"0x8D712161373C4B7\"",
  "name": "caselaw-idxr",
  "description": null,
  "dataSourceName": "caselaw-ds",
  "skillsetName": "caselaw-ss",
  "targetIndexName": "caselaw",
  "disabled": null,
  "schedule": null,
  "parameters": {
    "batchSize": 1,
    "maxFailedItems": null,
    "maxFailedItemsPerBatch": null,
    "base64EncodeKeys": null,
    "configuration": {
      "parsingMode": "jsonLines",
      "dataToExtract": "contentAndMetadata",
      "enableKnowledgeStoreAnnotationCache": true
    }
  },
  "fieldMappings": [],
  "outputFieldMappings": [
    {
      "sourceFieldName": "/document/casebody/data/opinions/*/text/pages/*/people/*",
      "targetFieldName": "people",
      "mappingFunction": null
    },
    {
      "sourceFieldName": "/document/casebody/data/opinions/*/text/pages/*/organizations/*",
      "targetFieldName": "orginizations",
      "mappingFunction": null


In [ ]:
# Create Datasource
datasourcename = "py-caselaw-ds"
datsource_def = {  
            'name': datasourcename,
            'description': 'Data source for cognitive search example',
            'type': 'azureblob',
            'credentials': {
                'connectionString': knowledge_store_key
            },
            'container': { 
                'name': 'enriched'
            }
        }
r = requests.post(construct_Url(search_service, "datasources", None, None, api_version), data=json.dumps(datsource_def),  headers=headers)
print(r)
res = r.json()
print(json.dumps(res, indent=2))

In [ ]:
# Create Skillset
# skillsetname = 'some-ss'
# skillset_def = {
         
#             "name": skillsetname,
#             "description": "A Python demo of the Caselaw Sample",
#             "skills": [
#             ],
#             "cognitiveServices": {
#                 "@odata.type": "#Microsoft.Azure.Search.CognitiveServicesByKey",
#                 "description": "mycogsvcs",
#                 "key": ""
#             },
#             "knowledgeStore": {
#                 "storageConnectionString": "",
#                 "projections": [
#                     {
#                         "tables": [
                           
#                         ],
#                         "objects": []
#                     }
#                 ]
#             }
#         }
# r = requests.put(construct_Url(search_service, "skillsets", skillsetname, None, api_version), data=json.dumps(skillset_def),  headers=headers)
# print(r)
# res = r.json()
# print(json.dumps(res, indent=2))

In [ ]:
# Create Index
indexname = "hotelreviews"
# index_def = {
#     "name": indexname,
#     "defaultScoringProfile": None,
#     "fields": [
#         {
#             "name": "id",
#             "type": "Edm.String",
#             "searchable": True,
#             "filterable": True,
#             "retrievable": True,
#             "sortable": True,
#             "facetable": True,
#             "key": True
#         }
#     ]
    
# }
# r = requests.post(construct_Url(search_service, "indexes", None, None, api_version), data=json.dumps(index_def),  headers=headers)
# print(r)
# res = r.json()
# print(json.dumps(res, indent=2))


In [ ]:
# Create Indexer

indexername = "UpdateCentroid-indexer"
indexer_def = {
            
            "name": indexername,
            "description": "Indexer for the updating the centroid field",
            "dataSourceName": datasourcename,
            "skillsetName": null,
            "targetIndexName": indexname,
            "disabled": None,
             "schedule": {
                "interval": "PT1H",
                "startTime": "0001-01-01T00:00:00Z"
            },
            "parameters": {
                "batchSize": 1,
                "maxFailedItems": None,
                "maxFailedItemsPerBatch": None,
                "base64EncodeKeys": None,
                "configuration": {
                    "parsingMode": "delimitedText",
                    "firstLineContainsHeaders": true,
                    "delimitedTextDelimiter": ",",
                    "delimitedTextHeaders": ""
                }
            },
            "fieldMappings": [],
            "outputFieldMappings": []
        }
r = requests.post(construct_Url(search_service, "indexers", None, None, api_version), data=json.dumps(indexer_def),  headers=headers)
print(r)
res = r.json()
print(json.dumps(res, indent=2))